In [21]:
# import all the various libraries
import pandas as pd
import json
from zipfile import ZipFile
from bs4 import BeautifulSoup
from azure.cosmos import database, exceptions, CosmosClient, PartitionKey
import os
import uuid

In [22]:
os.environ['COSMOS_URI'] = 'https://regtech-dev.documents.azure.com:443/'
os.environ['COSMOS_KEY'] = 'We7uwpThNrJhoEkcCRZ5JakoSqL5ZQAURrR8ZrqPBoHinQdF0GS3mCvpnmrCaKqb5oRCsAqUIUnNDtBQqJkjzg=='
# os.environ['COSMOS_URI'] = 'https://localhost:8081'
# os.environ['COSMOS_KEY'] = 'C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw=='
os.environ['DATABASE'] = 'ApplicibilityAssessmentDB'
os.environ['CONTAINER'] = 'AA_Filter_NC_CaptalNCPM'

In [23]:


container = os.environ['CONTAINER']
path = "D:\\GitHub\\sbsa_excel_to_cosmos_etl\\ApplicibilityAssessmentDB\\" + container + " - data.json"
json_url = open(path, encoding='utf-8')
json_data = json.load(json_url)

id=0
for i in range(len(json_data)):
    id +=1 
    t = json_data[i]
    t['id'] = str("0") + str(id) + str('_test')
    try:
        del t['_rid']
        del t['_self']
        del t['_etag']
        del t['_ts']
        del t['_attachments']
    except:
        continue

json_data



[{'id': '01_test',
  'Noun_Entity_Ruler': 'the Bank of Mozambique',
  'Noun_Entity_Ruler_Label': 'ORG',
  'Noun_Chunk_Phrase_Matcher_1': 'policy'}]

In [24]:
# instantiate the cosmos db client
url = os.environ['COSMOS_URI']
key = os.environ['COSMOS_KEY']
db = os.environ['DATABASE']

client = CosmosClient(url, credential=key)

partition_key = PartitionKey(path='/id', kind='Hash')

try:
    db = client.create_database(id=db)
except exceptions.CosmosResourceExistsError:
    db = client.get_database_client(database=db)

try:
    container = db.create_container(id=container, partition_key=partition_key)
except exceptions.CosmosResourceExistsError:
    container = db.get_container_client(container)

for i in range(len(json_data)):
    container.create_item(body=json_data[i])

CosmosHttpResponseError: (BadRequest) Message: {"Errors":["An unknown error occurred while processing this request. If the issue persists, please contact Azure Support: http://aka.ms/azure-support"]}
ActivityId: ed5aaf13-b313-4391-b4bc-9f8b2779e992, Request URI: /apps/39976179-6b0b-4d39-bf13-0586837e4e83/services/f3ba6613-f4a5-4788-96f3-271c79aa4066/partitions/a7fbf93a-aad6-4faa-9316-cf9d8ed642c5/replicas/132747338160660546p, RequestStats: , SDK: Microsoft.Azure.Documents.Common/2.14.0, Microsoft.Azure.Documents.Common/2.14.0
Code: BadRequest
Message: Message: {"Errors":["An unknown error occurred while processing this request. If the issue persists, please contact Azure Support: http://aka.ms/azure-support"]}
ActivityId: ed5aaf13-b313-4391-b4bc-9f8b2779e992, Request URI: /apps/39976179-6b0b-4d39-bf13-0586837e4e83/services/f3ba6613-f4a5-4788-96f3-271c79aa4066/partitions/a7fbf93a-aad6-4faa-9316-cf9d8ed642c5/replicas/132747338160660546p, RequestStats: , SDK: Microsoft.Azure.Documents.C